### MNIST 数据集

**mnist 数据集是一个非常出名的数据集，基本上很多网络都将其作为一个测试的标准，其来自美国国家标准与技术研究所, National Institute of Standards and Technology (NIST)。 训练集 (training set) 由来自 250 个不同人手写的数字构成, 其中 50% 是高中学生, 50% 来自人口普查局 (the Census Bureau) 的工作人员，一共有 60000 张图片。 测试集(test set) 也是同样比例的手写数字数据，一共有 10000 张图片。**
**每张图片大小是 28 x 28 的灰度图**

### 分类问题

#### ***softmax 函数示例如下:***

对于多分类问题，需要知道其属于每一类的概率，这个时候就需要 softmax 函数,对于网络的输出 $z_1, z_2, \cdots z_k$，我们首先对他们每个都取指数变成 $e^{z_1}, e^{z_2}, \cdots, e^{z_k}$，那么每一项都除以他们的求和，也就是

$$
z_i \rightarrow \frac{e^{z_i}}{\sum_{j=1}^{k} e^{z_j}}
$$

如果对经过 softmax 函数的所有项求和就等于 1，所以他们每一项都分别表示属于其中某一类的概率。

#### ***交叉熵***

交叉熵衡量两个分布相似性的一种度量方式，前面讲的二分类问题的 loss 函数就是交叉熵的一种特殊情况，交叉熵的一般公式为

$$
cross\_entropy(p, q) = E_{p}[-\log q] = - \frac{1}{m} \sum_{x} p(x) \log q(x)
$$

In [1]:
import numpy as np
import torch
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.datasets import mnist
from torchvision import datasets, transforms

In [2]:
# Hyperparameters #
batch_size = 64          # 批量化的大小 
learning_rate = 1e-2     # 学习率
num_epoches = 20         # 

### 数据的准备:
***下载数据集-装载数据集-建立迭代器***

In [3]:
# 定义数据标准化方法，在下载数据的时候，将数据进行批量标准化操作 #
def data_tf(x):
    x = np.array(x,dtype='float32')/255      # 除以最大值 (转成0-1之间)
    x = (x-0.5) / 0.5                        # 转成(-0.5,0.5)
    x = x.reshape((1,x.shape[0],x.shape[1])) # FCNN:转换成一维数据,直接拉平. CNN:如果是二维的,需要加上图片的深度:1
                                             ## ZQ：一般叫图像的通道数
    x = torch.from_numpy(x)                  # 将数据转成Tensor
    return x

# 导入数据集,直接转换成 <Numpy(ndarray)-标准化-Tensor>
train_set = datasets.MNIST(root='./data',train=True, transform=data_tf,download=True)
test_set  = datasets.MNIST(root='./data',train=False,transform=data_tf,download=True)

# 借助DataLoader取batch data，用于形成DataLoader的dataset不一定是tensor
# 可以是array，但是迭代DataLoader取出来的数一定是tensor，符合网络的直接输入需求
# 类似于将整个训练数据集进行切片,每batch size=64个切成一段,然后作为一批,喂给模型
train_data = DataLoader(train_set,batch_size=batch_size,shuffle=True)     # 确定好每次批量的大小，进行数据的自行填充
test_data  = DataLoader(test_set, batch_size=batch_size,shuffle=False)    # 60000个数据,分成64(8*8)大小,喂给模型
                                                                          ## ZQ：这里8*8指什么？
    
    
# 使用数据迭代器是非常有必要的，如果数据量太大，就无法一次将他们全部读入内存
# 所以需要使用 python 迭代器，每次生成一个批次的数据

In [4]:
#经过loader(装载)的数据,会自动分成 size=64,数据分为data和label两部分#
for i, data in enumerate(train_data):   # i 表示第几个元素，用 data 遍历train_loader
    if i == 0:
        print(type(data))
        print(data[0].shape)
        print(data[1])
# train_data里面的60000个数据,已经被分成以64为单位

<class 'list'>
torch.Size([64, 1, 28, 28])
tensor([6, 8, 4, 0, 7, 9, 2, 3, 6, 1, 7, 0, 0, 4, 0, 9, 4, 1, 2, 1, 0, 2, 0, 1,
        1, 8, 4, 6, 8, 8, 5, 1, 5, 4, 3, 5, 5, 4, 5, 0, 9, 6, 2, 2, 6, 3, 4, 6,
        5, 4, 3, 5, 0, 7, 7, 1, 5, 4, 1, 6, 6, 5, 0, 4])


In [5]:
for im,label in train_data:
    #将数据转化成Variable#
    im    = Variable(im)                   # 模型的输入数据: 四维的Tensor数据.一个im就是一个Batch
    label = Variable(label)
print(type(im))             # train_data里面的数据,类型是Tensor
print(im.shape)
print(type(label))          # train_data里面的数据,类型是Tensor
print(label.shape)

<class 'torch.Tensor'>
torch.Size([32, 1, 28, 28])
<class 'torch.Tensor'>
torch.Size([32])


***Question: 经过转换成Variabe()之后,为什么每个batch size的大小从64转换成了32***
* Answer: 因为print(im.shape)写在循环外，而train_data的总量不能被batch_size整除，相当于print出了一个余数的数量

In [6]:
# 对于x,y分开的数据,自行组装成Pair data,之后再进行Batch数据的生成 #
dataset1   = np.random.randn(300,1,64,64)
dataset2   = np.random.randn(300,1)
# MNIST数据集输入和输出数据已经是对应的paired dataset;
#但是有的x和y数据，需要在产生Batch数据之前,先进行自行构建paired dataset;
#使用torch.utils.data.TensorDataset函数，函数输入的量为tensor,由此构成 dataloder每次取出来是一个2元列表;
#两个元素分别是x batch和y batch
dataset_t  = torch.utils.data.TensorDataset(torch.FloatTensor(dataset1), torch.FloatTensor(dataset2))  # 参数是Tensor
# 在使用DataLoader之前,可以先将数据组装起来,形成数据对;再利用Bacth功能产生批量数据
dataloader = DataLoader(dataset_t, batch_size=20, shuffle=True)

for i, data in enumerate(dataloader):   #  i 表示第几个元素，用 data 遍历data_loader
    if i == 1:
        print(type(data))
        print(data[0].shape)
        print(data[1].shape)

<class 'list'>
torch.Size([20, 1, 64, 64])
torch.Size([20, 1])


### 模型的定义,供后续使用

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(  # in_chanels:输入通道数1;out_chanels:卷积核个数16;输入图片大小:28;卷积核大小:3
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0), #卷积层输出数据: (b, 16, 26, 26)
            nn.BatchNorm2d(16),       # 进行非线性变化之前,先进行标准化操作,使得数值趋近0-1,梯度变化明显,更高的向后传播
                                      ##ZQ：BN是变换到均值为0，标准差为1的分布，不是变换到0-1之间
            nn.ReLU())              
#             nn.MaxPool2d(kernel_size=2,stride=2)  #池化层输出数据: (b,)
            
        self.layer2 = nn.Sequential(  # in_chanels:输入通道数16;out_chanels:卷积核个数32;输入图片大小:28;卷积核大小:3;输出数据大小:(26-3+2*0)/1+1=24   
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0), #卷积层输出数据: (b, 32, 24, 24)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))  #池化层输出数据: (b, 32, 12, 12)
            
        self.layer3 = nn.Sequential(  
            # in_chanels:输入通道数32;out_chanels:卷积核个数64;输入图片大小:12;卷积核大小:3;输出数据大小:(12-3+2*0)/1+1=10   
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0), #卷积层输出数据: (b, 64, 10, 10)
            nn.BatchNorm2d(64),
            nn.ReLU())
#             nn.MaxPool2d(kernel_size=2,stride=2)  #池化层输出数据: (b,)
            
        self.layer4 = nn.Sequential(  # in_chanels:输入通道数10;out_chanels:卷积核个数128;输入图片大小:10;卷积核大小:3;输出数据大小:(10-3+2*0)/1+1=8   
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0), #卷积层输出数据: (b, 128, 8, 8)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))  #池化层输出数据: (b, 128, 4, 4)
            
        self.fc = nn.Sequential(
            nn.Linear(128*4*4,1024),              #进行全连接之前,将数据拉平,按照从后向前的顺序,列-行-深度 
            nn.ReLU(),
            nn.Linear(1024,128),
            nn.ReLU(),
            nn.Linear(128,10))
            
    def forward(self,x):
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            
            x = x.view(x.size(0),-1)  # 在进行FC-NN分析之前,要先将数据拉平
            x = self.fc(x)
            return x         

### 模型的训练与评价

In [8]:
#模型-优化器-损失函数
cnn = CNN()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)  
criterion = nn.CrossEntropyLoss()                                 # criterion=loss_function 分类问题:交叉熵(信息论)

In [17]:
# 模型的训练
train_loss= []
train_accu= []
eval_loss = []
eval_accu = []

for epoch in range(num_epoches):              # 训练模型的迭代次数,一个epoch模型会训练完train data里面的所有数据,这里会训练所有的数据20次
    
    #训练集上训练模型#
    train_loss_value = 0
    train_accu_value = 0
    cnn.train()                               # 转化成train的模式
    for im,label in train_data:
        #将数据转化成Variable#
        im    = Variable(im)                   # 模型的输入数据: 四维的Tensor数据.一个im是Batch里面的一段
        label = Variable(label)
        #前向传播#
        out   = cnn(im)
        loss = criterion(out,label)
        #反向传播#
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #计算loss和准确性#
        train_loss_value = train_loss_value + loss.item()
        _,pred = out.max(1)  #out输出,是对应的0-9的概率,按axis=1的方向求10个数最大值,得到最有可能的值,只需要返回数值,不需要位数,用_承接
        num_correct = (pred==label).sum().item()
        acc = num_correct/im.shape[0]
        train_accu_value  =  train_accu_value + acc
        
    train_loss.append(train_loss_value/len(train_data))    # 得到每一次运行完整个周期的平均的损失
    train_accu.append(train_accu_value/len(train_data))    # 得到每一次运行完整个周期的平均的准确性
    
    #测试集上测试模型#
    eval_loss_value = 0
    eval_accu_value = 0
    cnn.eval()                               # 转化成train的模式
    for im,label in test_data:
        #将数据转化成Variable#
        im    = Variable(im)                   # 模型的输入数据: 四维的Tensor数据.一个im是Batch里面的一段
        label = Variable(label)
        #前向传播#
        out   = cnn(im)
        loss = criterion(out,label)
        #计算loss和准确性#
        eval_loss_value = eval_loss_value + loss.item()
        _,pred = out.max(1)  #out输出,是对应的0-9的概率,按axis=1的方向求10个数最大值,得到最有可能的值,只需要返回数值,不需要位数,用_承接
        num_correct = (pred==label).sum().item()
        acc = num_correct/im.shape[0]
        eval_accu_value  =  eval_accu_value + acc
    eval_loss.append(eval_loss_value/len(test_data))    # 得到平均的损失
    eval_accu.append(eval_accu_value/len(test_data))    # 得到平均的准确性
    
    print("epoch: {}, Train Loss: {:.6f}, Train Accu: {:.6f}, Eval Loss: {:.6f}, Eval Accu{:.6f}"
          .format(epoch,train_loss[-1],train_accu[-1],eval_loss[-1],eval_accu[-1]))

epoch: 0; Train Loss: 0.036075; Train Accu: 0.991355; Eval Loss: 0.031534; Eval Accu0.991342
epoch: 1; Train Loss: 0.032861; Train Accu: 0.992204; Eval Loss: 0.060708; Eval Accu0.986564
epoch: 2; Train Loss: 0.030356; Train Accu: 0.992737; Eval Loss: 0.038484; Eval Accu0.991740
epoch: 3; Train Loss: 0.028040; Train Accu: 0.993520; Eval Loss: 0.043721; Eval Accu0.991342
epoch: 4; Train Loss: 0.038058; Train Accu: 0.991738; Eval Loss: 0.052009; Eval Accu0.989948
epoch: 5; Train Loss: 0.028446; Train Accu: 0.993903; Eval Loss: 0.042981; Eval Accu0.990645
epoch: 6; Train Loss: 0.031991; Train Accu: 0.993220; Eval Loss: 0.055004; Eval Accu0.989053
epoch: 7; Train Loss: 0.029899; Train Accu: 0.993737; Eval Loss: 0.042190; Eval Accu0.991541
epoch: 8; Train Loss: 0.024301; Train Accu: 0.995319; Eval Loss: 0.037682; Eval Accu0.992038
epoch: 9; Train Loss: 0.023791; Train Accu: 0.994803; Eval Loss: 0.055286; Eval Accu0.989849
epoch: 10; Train Loss: 0.032978; Train Accu: 0.993753; Eval Loss: 0.05